In [1]:
from __future__ import division, print_function, unicode_literals

# Data process pkg
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit
from decimal import *
getcontext().prec = 6

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

# Standardize features
from sklearn.preprocessing import StandardScaler

%matplotlib inline

# Preperation

In [2]:
# Read data
transaction = pd.read_csv("https://s3.amazonaws.com/pernalonga/transaction_table.csv",header=0)
product = pd.read_csv("https://s3.amazonaws.com/pernalonga/product_table.csv",header=0)
week = pd.read_csv("https://s3.amazonaws.com/pernalonga/week.csv", header=0)

In [3]:
# Concat the week column calculated from R to the transaction table
transaction = pd.concat([transaction, week], axis = 1)
transaction.rename(columns={'x':'week'}, inplace=True)

In [4]:
# Convert transaction date to date format
transaction['tran_dt'] = pd.to_datetime(transaction['tran_dt'], format='%Y-%m-%d')

In [5]:
# Create a new column with new transaction_id
transaction['transaction_id'] = transaction['tran_dt'].dt.strftime('%Y%m%d') + transaction['cust_id'].astype(str) \
+ transaction['store_id'].astype(str)

In [6]:
# Create a new column with year
transaction['year'] = transaction['tran_dt'].dt.year

In [7]:
# First 5 rows of transaction table
transaction.head()

,cust_id,tran_id,tran_dt,store_id,prod_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price,week,transaction_id,year
0,139662,2.017110e+18,2017-11-03,584,145519008,CT,2.89,4.0,0.00,0,2.89,0.7225,44,20171103139662584,2017
1,799924,2.017111e+18,2017-11-12,349,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,45,20171112799924349,2017
2,1399898,2.017102e+18,2017-10-21,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,42,201710211399898684,2017
3,1399898,2.017111e+18,2017-11-11,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,45,201711111399898684,2017
4,1399898,2.017121e+18,2017-12-05,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,49,201712051399898684,2017


In [8]:
# Found 8 instances with negative paid amount
transaction[transaction.tran_prod_paid_amt<0]  

,cust_id,tran_id,tran_dt,store_id,prod_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price,week,transaction_id,year
647735,93409897,2.017080e+18,2017-08-03,340,357541011,CT,0.55,1.0,-1.10,1,-0.55,0.55,31,2017080393409897340,2017
6816009,73479594,2.016100e+18,2016-10-05,341,999241421,CT,16.90,1.0,-16.98,2,-0.08,16.90,40,2016100573479594341,2016
6816010,73479594,2.016100e+18,2016-10-05,342,999241421,CT,33.80,2.0,-33.96,2,-0.16,16.90,40,2016100573479594342,2016
7730846,40099908,2.016052e+18,2016-05-22,344,999250092,CT,1.59,1.0,-1.60,1,-0.01,1.59,20,2016052240099908344,2016
9135107,51749812,2.016092e+18,2016-09-19,325,999264989,CT,0.30,1.0,-0.49,3,-0.19,0.30,38,2016091951749812325,2016
10758296,42509966,2.016062e+18,2016-06-23,984,999295518,CT,3.59,1.0,-3.94,1,-0.35,3.59,25,2016062342509966984,2016
17069555,16339676,2.016033e+18,2016-03-26,576,999436833,KG,5.49,1.0,-6.90,1,-1.41,5.49,12,2016032616339676576,2016
18424578,7869780,2.016082e+18,2016-08-17,988,999476721,CT,3.29,1.0,-3.30,2,-0.01,3.29,33,201608177869780988,2016


In [9]:
# Drop transactions with negative paid amount
transaction=transaction.loc[transaction['tran_prod_paid_amt']>=0]

In [10]:
# Change tran_prod_discount_amt into positive
transaction['discount_amt'] = transaction['tran_prod_discount_amt'].apply(abs)/transaction['tran_prod_sale_qty']

In [11]:
# Number of products sold in two years
transaction[['year', 'prod_id']].groupby(['year']).nunique()

,year,prod_id
year,,
2016,1,9742
2017,1,10539


In [12]:
# Since some of the products only appears in 2017, we decided to consider only 2017 transactions
# Otherwise, some of the products will have shorter time period data, not a valid comparison
trans2017 = transaction.loc[transaction.year == 2017,]

In [13]:
# Merge product information with transaction data
trans2017_df = trans2017.merge(product[['prod_id', 'category_id']], on = 'prod_id')

In [14]:
trans2017_df.head()

,cust_id,tran_id,tran_dt,store_id,prod_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price,week,transaction_id,year,discount_amt,category_id
0,139662,2.017110e+18,2017-11-03,584,145519008,CT,2.89,4.0,0.00,0,2.89,0.7225,44,20171103139662584,2017,0.0000,95854
1,799924,2.017111e+18,2017-11-12,349,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,45,20171112799924349,2017,0.3625,95854
2,1399898,2.017102e+18,2017-10-21,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,42,201710211399898684,2017,0.3625,95854
3,1399898,2.017111e+18,2017-11-11,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,45,201711111399898684,2017,0.3625,95854
4,1399898,2.017121e+18,2017-12-05,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,49,201712051399898684,2017,0.3625,95854


# Prepare Big Table for Demand Function

In [15]:
# Calcualte each product weekly prod_unit_price and tran_prod_paid_amt
# Possible to have multiple price change or discount price within a week (take mean)
prod_info = trans2017_df[['category_id', 'prod_id', 'store_id', 'week', 'prod_unit_price', 'discount_amt']].\
groupby(['category_id', 'prod_id', 'store_id', 'week']).mean().reset_index()

In [16]:
# Calculate category sales per week per store
cat_sales = trans2017_df[['category_id', 'store_id', 'week', 'tran_prod_paid_amt']].groupby(['category_id', 'store_id', 'week']).\
sum().reset_index()

In [17]:
cat_sales.head()

,category_id,store_id,week,tran_prod_paid_amt
0,95052,137,1,8.47
1,95052,137,2,3.52
2,95052,137,3,1.98
3,95052,137,4,4.14
4,95052,137,5,3.74


In [18]:
# Merge seasonality info with prod_info
table = prod_info.merge(cat_sales, on = ['category_id', 'store_id', 'week'])

In [19]:
table.rename(columns = {'tran_prod_paid_amt': 'cat_sales'}, inplace = True)

In [20]:
table.head()

,category_id,prod_id,store_id,week,prod_unit_price,discount_amt,cat_sales
0,95052,999168023,137,51,1.29,0.3000,0.99
1,95052,999168023,137,52,0.99,0.0000,1.98
2,95052,999168023,143,22,1.19,0.0000,1.19
3,95052,999168023,148,29,0.99,0.0300,7.68
4,95052,999397567,148,29,0.44,0.0425,7.68


## Filter

### Filter out the products that contribute 90% sales

In [21]:
# Calculate the number of products
product2017_sales = trans2017_df[['prod_id','tran_prod_paid_amt']].groupby('prod_id').sum().reset_index().\
sort_values('tran_prod_paid_amt',ascending=False)

product2017_sales['percentage']=product2017_sales['tran_prod_paid_amt']/33202391.5*100
product2017_sales['percentage'][0:4800].sum()

90.2324118429834

In [22]:
# Filter the table
prod_trans = transaction.loc[transaction.prod_id.isin(product2017_sales.prod_id)]

In [23]:
# Number of transactions after the filter
prod_trans.shape

(29408151, 16)

### Substitute

In [25]:
# Create the table of each product with the other products purchase together
trans_merged = prod_trans[['transaction_id', 'prod_id']].merge(product[['prod_id', 'subcategory_id']], on = 'prod_id')

In [26]:
# Count the number that the product show up
sub_count = trans_merged[['transaction_id', 'subcategory_id', 'prod_id']].groupby(['subcategory_id', 'prod_id']).\
size().rename('count').reset_index()

In [27]:
sub_count.head()

,subcategory_id,prod_id,count
0,90265,999248083,324
1,90265,999351167,210
2,90266,999262629,245
3,90270,999143864,145
4,90287,999153808,484


In [28]:
# sort by least sales 
least_sales = sub_count.sort_values(['subcategory_id', 'count'], ascending = True).groupby(['subcategory_id']).head(1)
least_sales.rename(columns = {'prod_id': 'sub_id'}, inplace = True)

In [29]:
least_sales.head()

,subcategory_id,sub_id,count
1,90265,999351167,210
2,90266,999262629,245
3,90270,999143864,145
8,90287,999156313,93
14,90289,999173753,377


In [30]:
# Merge back to the original table to get the substitute list
prod_sub = product[['subcategory_id', 'prod_id']].loc[product.prod_id.isin(product2017_sales.prod_id),]
sub = prod_sub.merge(least_sales, on = ['subcategory_id'], how = 'left')

In [31]:
product[['subcategory_id', 'prod_id']].loc[product.prod_id.isin(product2017_sales.prod_id),].shape

(10536, 2)

In [32]:
sub.head()

,subcategory_id,prod_id,sub_id,count
0,93970,145519008,234423012,315
1,93970,145519009,234423012,315
2,93970,145519010,234423012,315
3,93970,145519011,234423012,315
4,93970,145519012,234423012,315


### Merge back to the big table

In [21]:
product_2017_sub=trans2017_df
product_2017_sub.rename(columns={'prod_id':'sub_id','prod_unit_price':'sub_price'}, inplace=True)

In [23]:
table1 = pd.merge(table,sub[['prod_id','sub_id']],on=['prod_id'])